In [28]:
import json
import numpy as np
from torchvision import datasets, transforms

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick to prevent overflow
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

# Initialize neuron functions dynamically
def create_neuron(weights, bias, activation, is_final_layer=False):
    activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
    
    def neuron(inputs):
        z = np.dot(weights, inputs) + bias
        return z if is_final_layer else activation_func(z)
    
    neuron.weights = weights  # Store weights for debugging
    neuron.bias = bias  # Store bias for debugging
    neuron.activation = activation  # Store activation type
    return neuron

# Build network dynamically from JSON structure
def build_network(json_data):
    layers = []  # Store network layers
    
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))  # Ensure correct order
    
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        layer_neurons = []
        
        for node in layer_info['nodes']:
            weights = np.array(node['weights'])
            biases = np.array(node['biases'])
            activation = node['activation']
            
            neuron_func = create_neuron(weights, biases, activation, is_final_layer=(i == len(sorted_layers) - 1))
            layer_neurons.append(neuron_func)
        
        layers.append(layer_neurons)
    
    return layers

# Forward pass with Debugging
def forward_pass(layers, input_data):
    x = np.array(input_data)
    #print("Input to network:", x[:10])  # Debugging: Check first 10 inputs
    
    # Pass through each layer
    for i, layer in enumerate(layers):
        raw_output = np.array([np.dot(neuron.weights, x) + neuron.bias for neuron in layer])  # Pre-activation
        #print(f"Layer {i} raw output:", raw_output[:10])  # Debugging: Before activation
        
        x = np.array([neuron(x) for neuron in layer])
        #print(f"Layer {i} output after activation:", x[:10])  # Debugging: After activation
        
        # Apply softmax only at final layer
        if i == len(layers) - 1:
            x = softmax(x)
            #print("Final softmax output:", x)  # Debugging final output
    
    return np.argmax(x)  # Return predicted class

# Load network
data = load_network("node_based_model.json")
network = build_network(data)
print(len(network[0]))
print(len(network[1]))

# Load MNIST dataset with correct normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test on MNIST images
for i in range(10):  # Testing on first 10 images
    image, label = mnist_test[i]
    image = image.view(-1).numpy()  # Flatten 28x28 image to a 784 input vector
    
    predicted_class = forward_pass(network, image)
    print(f"Image {i}: Label = {label}, Predicted = {predicted_class}")


128
10
Image 0: Label = 7, Predicted = 7
Image 1: Label = 2, Predicted = 2
Image 2: Label = 1, Predicted = 1
Image 3: Label = 0, Predicted = 0
Image 4: Label = 4, Predicted = 4
Image 5: Label = 1, Predicted = 1
Image 6: Label = 4, Predicted = 4
Image 7: Label = 9, Predicted = 9
Image 8: Label = 5, Predicted = 6
Image 9: Label = 9, Predicted = 9


In [10]:
from pcomp.activation_functions import ACTIVATIONS
from pcomp.neuron import Neuron
from pcomp.layer import Layer

In [12]:
import json
import numpy as np
import re
from torchvision import datasets, transforms

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Build network dynamically from JSON structure
def build_network(json_data):
    layers = []  # Store network layers
    
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(re.findall(r'\d+', x)[-1]))  # Ensure correct order
    
    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        neurons = [Neuron(node['weights'], node['biases'], node['activation'], is_final_layer=(i == len(sorted_layers) - 1)) for node in layer_info['nodes']]
        layers.append(Layer(neurons, is_final_layer=(i == len(sorted_layers) - 1)))
    
    return layers

# Forward pass with Debugging
def forward_pass(layers, input_data):
    x = np.array(input_data)
    print("Input to network:", x[:10])  # Debugging: Check first 10 inputs
    
    # Pass through each layer
    for i, layer in enumerate(layers):
        x = layer.forward(x)  # Call Layer forward pass
        print(f"Layer {i} output after activation:", x[:10])  # Debugging: After activation
    
    return np.argmax(x)  # Return predicted class

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset with correct normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test on MNIST images
for i in range(10):  # Testing on first 10 images
    image, label = mnist_test[i]
    image = image.view(-1).numpy()  # Flatten 28x28 image to a 784 input vector
    
    predicted_class = forward_pass(network, image)
    print(f"Image {i}: Label = {label}, Predicted = {predicted_class}")

Input to network: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
Layer 0 output after activation: [0.         1.02855658 2.23111197 0.         3.52626708 0.
 3.77270466 0.         4.32164268 0.        ]
Layer 1 output after activation: [1.63366778e-08 2.05432143e-10 1.23416420e-06 1.92995192e-04
 2.74237134e-10 1.96794985e-07 4.10642952e-11 9.99804679e-01
 6.17640000e-07 2.60397545e-07]
Image 0: Label = 7, Predicted = 7
Input to network: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
Layer 0 output after activation: [1.8688848  0.         0.18511521 0.         0.         0.
 1.54902889 0.         0.07973161 0.        ]
Layer 1 output after activation: [1.29415276e-05 1.15765649e-04 9.99762976e-01 7.60362304e-05
 4.84269700e-11 1.56052707e-06 1.49202731e-05 7.87612529e-09
 1.57922838e-05 2.57884985e-12]
Image 1: Label = 2, Predicted = 2
Input to network: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
Layer 0 output after activation: [0.26478969 0.03137161 2.17695347 4.39132531 0.         0.
 0.       

In [16]:
import json
import numpy as np
from torchvision import datasets, transforms

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x))  # Stability trick to prevent overflow
    return e_x / e_x.sum()

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

# Global storage to simulate Redis
storage = {}

class Neuron:
    def __init__(self, weights, bias, activation, layer_id, neuron_id, is_final_layer=False):
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.layer_id = layer_id
        self.neuron_id = neuron_id
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        # Ensure the number of inputs matches the weight dimension
        input_values = np.array([storage[inp] for inp in inputs if inp in storage])

        # if input_values.shape[0] != self.weights.shape[0]:
        #     raise ValueError(f"Neuron {self.layer_id}_{self.neuron_id} received {input_values.shape[0]} inputs but expected {self.weights.shape[0]}")

        # Compute activation
        z = np.dot(self.weights, input_values) + self.bias
        output = z if self.is_final_layer else self.activation_func(z)
        
        # Store output in the storage dictionary
        storage[f"{self.layer_id}_{self.neuron_id}"] = output
        return output

class Layer:
    def __init__(self, layer_id, neurons):
        self.layer_id = layer_id
        self.neurons = neurons

    def forward(self, input_keys):
        inputs = [key for key in input_keys if key in storage]  # Ensure valid keys
        outputs = [neuron.forward(inputs) for neuron in self.neurons]
        return outputs

# Build network dynamically from JSON
def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))  # Ensure correct order

    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        layer_neurons = []

        for j, node in enumerate(layer_info['nodes']):
            neuron = Neuron(
                weights=node['weights'],
                bias=node['biases'],
                activation=node['activation'],
                layer_id=layer_name,
                neuron_id=j,
                is_final_layer=(i == len(sorted_layers) - 1)
            )
            layer_neurons.append(neuron)
        
        layers.append(Layer(layer_name, layer_neurons))

    return layers

# Forward pass using storage dictionary
def forward_pass(layers, input_data):
    storage.clear()  # Clear storage before processing new image

    # Initialize storage with input values
    for i, val in enumerate(input_data):
        storage[f"input_{i}"] = val

    # Process each layer
    input_keys = [f"input_{i}" for i in range(len(input_data))]

    for layer in layers:
        layer.forward(input_keys)
        input_keys = [f"{layer.layer_id}_{j}" for j in range(len(layer.neurons))]

    # Retrieve final layer outputs
    final_outputs = np.array([storage[f"{layers[-1].layer_id}_{j}"] for j in range(len(layers[-1].neurons))])
    
    # Apply softmax only at the final layer
    final_outputs = softmax(final_outputs)
    
    return np.argmax(final_outputs)  # Return predicted class

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset with correct normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Test on MNIST images
for i in range(10):  # Testing on first 10 images
    image, label = mnist_test[i]
    image = image.view(-1).numpy()  # Flatten 28x28 image to a 784 input vector
    
    predicted_class = forward_pass(network, image)
    print(f"Image {i}: Label = {label}, Predicted = {predicted_class}")


Image 0: Label = 7, Predicted = 7
Image 1: Label = 2, Predicted = 2
Image 2: Label = 1, Predicted = 1
Image 3: Label = 0, Predicted = 0
Image 4: Label = 4, Predicted = 4
Image 5: Label = 1, Predicted = 1
Image 6: Label = 4, Predicted = 4
Image 7: Label = 9, Predicted = 9
Image 8: Label = 5, Predicted = 6
Image 9: Label = 9, Predicted = 9
